In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# # Load environment variables
# from dotenv import load_env
# load_env()

import os
import yaml
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, PDFSearchTool, WebsiteSearchTool, ScrapeWebsiteTool, DirectoryReadTool, FileReadTool, CodeInterpreterTool
from langchain_community.tools import TavilySearchResults
from langchain_community.utilities.alpha_vantage import AlphaVantageAPIWrapper
import DataAcquisition, DataProcessing, BuildTradingStrategy, BackTest, MarketAnalysis
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.utilities import SQLDatabase


In [2]:
import os
import yaml
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
# from langchain.chat_models import init_chat_model

# llm = init_chat_model(""llama3-8b-8192", model_provider="groq"")

In [5]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']

In [6]:
data_agent = Agent(
    config=agents_config['Data Acquisition Agent'],
    verbose=True,
            llm=llm,
            tools=[
                ScrapeWebsiteTool(),
                WebsiteSearchTool(),
                #AlphaVantageAPIWrapper(),
                #TavilySearchResults(max_results=10),
                SerperDevTool(),
                DirectoryReadTool(),
                FileReadTool(),
                CodeInterpreterTool(),
            ]
)

asset_class = "stocks"
instrument_name = "AAPL"
market = "US"

acquisition_tasks = DataAcquisition.DataAcquisitionTasks()

task_scrape_news = acquisition_tasks.scrape_news_sentiment(data_agent, asset_class, instrument_name, market)
task_ingest_actions = acquisition_tasks.ingest_corporate_actions(data_agent, asset_class, instrument_name, market)



NameError: name 'llm' is not defined

In [7]:
import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Verify that the Groq API key is available
groq_api_key = os.getenv("GROQ_API_KEY")
if not groq_api_key:
    raise EnvironmentError("GROQ_API_KEY not set in .env file. Please add your Groq API key.")

# Initialize the Groq-backed Llama model
from langchain.chat_models import init_chat_model
llm = init_chat_model("groq/llama3-8b-8192",model_provider="groq")

# Now initialize your agent using the Groq-backed model (and your wrapped tools)
from crewai import Agent, Crew
from DataAcquisition import DataAcquisitionTasks
from crewai_tools import ScrapeWebsiteTool, WebsiteSearchTool, SerperDevTool, DirectoryReadTool, FileReadTool, CodeInterpreterTool
from langchain_community.tools import TavilySearchResults
#from FinBot.AlphaVantageTool import AlphaVantageTool  # Wrapped tool for AlphaVantage

data_agent = Agent(
    config=agents_config['Data Acquisition Agent'],
    verbose=True,
    llm=llm,
    tools=[
        ScrapeWebsiteTool(),
        WebsiteSearchTool(),
        #AlphaVantageTool(),  # Using the wrapped tool
        #TavilySearchResults(max_results=10),
        SerperDevTool(),
        DirectoryReadTool(),
        FileReadTool(),
        CodeInterpreterTool(),
    ]
)

# Define common parameters for your data acquisition tasks
asset_class = "stocks"
instrument_name = "AAPL"
market = "US"

# Instantiate your DataAcquisitionTasks manager and create tasks
acquisition_tasks = DataAcquisitionTasks()
task_fetch_ohlc = acquisition_tasks.fetch_multi_frequency_ohlc(data_agent, asset_class, instrument_name, market)
task_stream_ticks = acquisition_tasks.stream_real_time_ticks(data_agent, asset_class, instrument_name, market)
task_orderbook = acquisition_tasks.collect_orderbook_option_data(data_agent, asset_class, instrument_name, market)
task_scrape_news = acquisition_tasks.scrape_news_sentiment(data_agent, asset_class, instrument_name, market)
task_corporate = acquisition_tasks.ingest_corporate_actions(data_agent, asset_class, instrument_name, market)
task_economic = acquisition_tasks.fetch_economic_alt_data(data_agent, asset_class, market)
task_compliance = acquisition_tasks.validate_and_log_compliance(data_agent, asset_class, market)

# Create the Crew and register tasks
crew = Crew(
    agents=[data_agent],
    tasks=[
        task_fetch_ohlc,
        task_stream_ticks,
        task_orderbook,
        task_scrape_news,
        task_corporate,
        task_economic,
        task_compliance
    ]
)

# Kickoff the tasks via CrewAI orchestration
crew.kickoff()

# Agent: Market Data Retrieval Specialist
## Task: [DATA ACQUISITION]
Collect OHLC data for stocks instrument AAPL in the US market across multiple frequencies (e.g., 5m, 15m, 1h, daily).

Guidelines:
- Use asset-class and market appropriate data sources (e.g., AlphaVantage for stocks, Binance for crypto).
- Respect API rate limits and include robust error handling.
- Store all raw data in a shared repository for downstream processing:
  Path: /data/raw/us/stocks/AAPL/ohlc_{FREQUENCY}_{YYYY-MM}.parquet

The output of this task will be used as the input for the data preprocessing pipeline.



# Agent: Market Data Retrieval Specialist
## Thought: Thought: I need to collect OHLC data for AAPL stock in the US market across multiple frequencies.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.alphavantage.co/documentation/#api\"}"
## Tool Output: 

API Documentation | Alpha Vantage
Table of Contents
Core Stock APIs
Intraday Trending
Daily
Daily Adjusted 

2025-02-11 11:39:17,263 - 8650263552 - llm.py-llm:309 - ERROR: LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Request too large for model `llama3-8b-8192` in organization `org_01jkr9zm6aesrt04dhcz0sf0ng` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Requested 142902, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.","type":"tokens","code":"rate_limit_exceeded"}}





LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Request too large for model `llama3-8b-8192` in organization `org_01jkr9zm6aesrt04dhcz0sf0ng` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Requested 142902, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.","type":"tokens","code":"rate_limit_exceeded"}}



RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Request too large for model `llama3-8b-8192` in organization `org_01jkr9zm6aesrt04dhcz0sf0ng` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Requested 142902, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.","type":"tokens","code":"rate_limit_exceeded"}}
